## Quantum machine learning: Classification problem

In [26]:
#pip install pandas
#pip install scikit-learn

import cudaq
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import scipy

In [27]:
mnist = fetch_openml('mnist_784', version=1, cache=True)
data = mnist['data']
labels = np.array(mnist['target'], dtype=np.int8)
        
labels_zero = labels[labels==0] 
labels_one = labels[labels==1] 
binary_labels = np.hstack((labels_zero, labels_one))
digits_zero = data[labels==0]
digits_one = data[labels==1]
binary_digits = np.vstack((digits_zero, digits_one))
        
pca = PCA(n_components=6)
sc = StandardScaler()
binary_digits = sc.fit_transform(binary_digits)
data = pca.fit_transform(binary_digits)

# Normalize data (Prepare data for angle encoding)
data = ((data-np.min(data))/(np.max(data)-np.min(data)))

x_train, x_test, y_train, y_test = train_test_split(data, binary_labels, test_size=0.2)

x_train=x_train[0:500]
x_test=x_test[0:200]
y_train=y_train[0:500]
y_test=y_test[0:200]

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(y_train[0:10])

(500, 6) (200, 6) (500,) (200,)
[0 0 0 1 0 1 0 0 0 1]


In [28]:
#labels are -1 and 1

y_train=y_train*2-1
y_test=y_test*2-1

print(y_train[0:10])


[-1 -1 -1  1 -1  1 -1 -1 -1  1]


In [29]:
cudaq.set_target("nvidia")

@cudaq.kernel
def encoding(qubit:cudaq.qview, angles:list[float]):
    # Define gates and the qubits they act upon.

    qubit_num=qubit.size()
    for i in range(qubit_num):
        ry(angles[i], qubit[i])

@cudaq.kernel
def training_layer(qubit:cudaq.qview, alpha:list[float]):

    qubit_num=qubit.size()

    count=0
    for i in range(qubit_num):
        u3(alpha[count],alpha[count+1],alpha[count+2], qubit[i])
        count+=3
    
    for i in range(qubit_num-1):
        x.ctrl(qubit[i], qubit[i+1])
    
    x.ctrl(qubit[qubit_num-1], qubit[0])

    for i in range(qubit_num):
        u3(alpha[count],alpha[count+1],alpha[count+2], qubit[i])
        count+=3

    for i in range(qubit_num//2):
        x.ctrl(qubit[i], qubit[i+2])
    
    for i in range(qubit_num-1, qubit_num//2-1, -1):
        x.ctrl(qubit[i], qubit[i-2])
    
@cudaq.kernel
def qnn_kernel(qubit_count:int, data: list[float], alpha:list[float]):
    # Allocate a qubit that is initialised to the |0> state.
    qubit = cudaq.qvector(qubit_count)

    # encoding data
    encoding(qubit,data)
    
    # training layer
    training_layer(qubit,alpha)

# Our hamiltonian will be the Z expectation value of our qubit.
ham = cudaq.spin.z(0)

In [30]:
qubit_count = x_train.shape[1]

np.random.seed(42)
alpha=np.random.normal(loc=0, scale=1, size=3*(qubit_count*2))
alpha=alpha.tolist()

bias=[0.0]
alpha=alpha+bias

print(alpha)
n=len(alpha)
print(alpha[n-1])

[0.4967141530112327, -0.13826430117118466, 0.6476885381006925, 1.5230298564080254, -0.23415337472333597, -0.23413695694918055, 1.5792128155073915, 0.7674347291529088, -0.4694743859349521, 0.5425600435859647, -0.46341769281246226, -0.46572975357025687, 0.24196227156603412, -1.913280244657798, -1.7249178325130328, -0.5622875292409727, -1.0128311203344238, 0.3142473325952739, -0.9080240755212109, -1.4123037013352915, 1.465648768921554, -0.22577630048653566, 0.06752820468792384, -1.4247481862134568, -0.5443827245251827, 0.11092258970986608, -1.1509935774223028, 0.37569801834567196, -0.600638689918805, -0.2916937497932768, -0.6017066122293969, 1.8522781845089378, -0.013497224737933921, -1.0577109289559004, 0.822544912103189, -1.2208436499710222, 0.0]
0.0


In [31]:
def cost(main_kernel, ham,qubit_count,angles,alpha):
    expectation_value = cudaq.observe(main_kernel, ham, qubit_count,angles,alpha).expectation()
    return expectation_value

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

def loss(alpha, main_kernel, ham, qubit_count, x_train, y_train):
    pred=[]

    param=alpha[0:n-2]
    for image in range (len(x_train)):
    
        eig=cost(main_kernel, ham, qubit_count, x_train[image], param)
        pred.append(eig+alpha[n-1])

    return square_loss(y_train,pred)

def variational_classifier(alpha,main_kernel,ham,qubit_count,data_img):
    
    param=alpha[0:n-2]
    
    pred=cost(main_kernel, ham, qubit_count,data_img, param)+alpha[n-1]
    
    return pred

In [32]:

batch_size = 10
batch_index = np.random.randint(0, len(x_train), (batch_size,))
X_batch = x_train[batch_index]
Y_batch = y_train[batch_index]


result_opt=scipy.optimize.minimize(loss, alpha, method='L-BFGS-B', jac='3-point', args=(qnn_kernel, ham, qubit_count, X_batch, Y_batch), tol=1e-8)

alpha=result_opt.x

predictions = [np.sign(variational_classifier(alpha,qnn_kernel,ham,qubit_count,x_train[i])) for i in range(len(x_train))]

   
acc_train = accuracy(y_train, predictions)

c_train=loss(alpha, qnn_kernel, ham, qubit_count, x_train, y_train)


print('Training data')
print(f'Cost: {c_train}, Accuracy: {np.round(acc_train, 2) * 100}%')

pred = [np.sign(variational_classifier(alpha,qnn_kernel,ham,qubit_count,x_test[i])) for i in range(len(x_test))]
   
acc_test = accuracy(y_test, pred)

c_test=loss(alpha, qnn_kernel, ham, qubit_count, x_test, y_test)

print('Test data')
print(f'Cost: {c_test}, Accuracy: {np.round(acc_test, 2) * 100}%')


Training data
Cost: 0.4417883210609934, Accuracy: 99.0%
Test data
Cost: 0.425547898720139, Accuracy: 100.0%


In [33]:
pd.DataFrame((pred, y_test), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,-1.0
1,-1.0,-1.0
2,1.0,1.0
3,-1.0,-1.0
4,-1.0,-1.0
...,...,...
195,1.0,1.0
196,1.0,1.0
197,1.0,1.0
198,-1.0,-1.0
